In [ ]:
import instructor
import openai
from qdrant_client import QdrantClient
from pydantic import BaseModel, Field

In [ ]:
prompt = """
You are a helpful assistant. Return the answer to the question.
Question: What is your name?
"""

In [ ]:
openai_response = openai.chat.completions.create(
    model="gpt-4.1-mini",
    messages=[
        {"role": "system", "content": prompt}
    ],
    temperature=0.0
)
print(openai_response.choices[0].message.content)

In [ ]:
openai_response

## Add insturctor (Structured Output)

In [ ]:
client = instructor.from_openai(openai.OpenAI())

In [ ]:
class RAGGeneretionResponce(BaseModel):
    answer: str = Field(description="Answer to the question")


In [ ]:
response, raw_responce = client.chat.completions.create_with_completion(
    model="gpt-4.1-mini",
    messages=[
        {"role": "system", "content": prompt}
    ],
    temperature=0.0,
    response_model=RAGGeneretionResponce
)

In [ ]:
response

In [ ]:
raw_responce

In [ ]:
class RAGGeneretionResponce(BaseModel):
    answer: str = Field(description="Answer to the question")
    reasoning: str = Field(description="Reasoning for the answer")


In [ ]:
response, raw_responce = client.chat.completions.create_with_completion(
    model="gpt-4.1-mini",
    messages=[
        {"role": "system", "content": prompt}
    ],
    temperature=0.0,
    response_model=RAGGeneretionResponce
)

In [ ]:
response

In [ ]:
raw_responce

### RAG Example

In [ ]:
class RAGGeneretionResponce(BaseModel):
    answer: str = Field(description="Answer to the question")


In [ ]:
def get_embedding(text, model="text-embedding-3-small"):
    response = openai.embeddings.create(
        input=text,
        model=model,
    )

    return response.data[0].embedding


def retrieve_data(query, qdrant_client, k=5):

    query_embedding = get_embedding(query)

    results = qdrant_client.query_points(
        collection_name="Amazon-items-collection-00",
        query=query_embedding,
        limit=k,
    )

    retrieved_context_ids = []
    retrieved_context = []
    similarity_scores = []
    retrieved_context_ratings = []

    for result in results.points:
        retrieved_context_ids.append(result.payload["parent_asin"])
        retrieved_context.append(result.payload["description"])
        retrieved_context_ratings.append(result.payload["average_rating"])
        similarity_scores.append(result.score)

    return {
        "retrieved_context_ids": retrieved_context_ids,
        "retrieved_context": retrieved_context,
        "retrieved_context_ratings": retrieved_context_ratings,
        "similarity_scores": similarity_scores,
    }


def process_context(context):

    formatted_context = ""

    for id, chunk, rating in zip(context["retrieved_context_ids"], context["retrieved_context"], context["retrieved_context_ratings"]):
        formatted_context += f"- ID: {id}, rating: {rating}, description: {chunk}\n"

    return formatted_context


def build_prompt(preprocessed_context, question):

    prompt = f"""
You are a shopping assistant that can answer questions about the products in stock.

You will be given a question and a list of context.

Instructtions:
- You need to answer the question based on the provided context only.
- Never use word context and refer to it as the available products.

Context:
{preprocessed_context}

Question:
{question}
"""

    return prompt


def generate_answer(prompt):

    response, raw_responce = client.chat.completions.create_with_completion(
        model="gpt-4.1-mini",
        messages=[{"role": "system", "content": prompt}],
        temperature=0,
        response_model=RAGGeneretionResponce
    )

    return response


def rag_pipeline(question, qdrant_client, top_k=5):

    retrieved_context = retrieve_data(question, qdrant_client, top_k)
    preprocessed_context = process_context(retrieved_context)
    prompt = build_prompt(preprocessed_context, question)
    answer = generate_answer(prompt)

    final_result = {
        "datamodel": answer,
        "answer": answer.answer,
        "question": question,
        "retrieved_context_ids": retrieved_context["retrieved_context_ids"],
        "retrieved_context": retrieved_context["retrieved_context"],
        "similarity_scores": retrieved_context["similarity_scores"]
    }

    return final_result

In [ ]:
quadrant_client = QdrantClient(url="http://localhost:6333")

In [ ]:
output = rag_pipeline("Can I get a charger cable?", quadrant_client)

In [ ]:
output

In [ ]:
print(output["answer"])